<a href="https://colab.research.google.com/github/Migrice/Master2-Projects/blob/main/RNN_sentiment_analysis_with_word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from zipfile import ZipFile 
  
# spécifiant le nom du fichier zip
file = "Sentiment analysis dataset.zip"
  
# ouvrir le fichier zip en mode lecture
with ZipFile(file, 'r') as zip: 
    # afficher tout le contenu du fichier zip
    zip.printdir() 
  
    # extraire tous les fichiers
    print('extraction...') 
    zip.extractall() 
    print('Terminé!')

File Name                                             Modified             Size
Sentiment analysis dataset/                    2022-11-22 09:41:54            0
Sentiment analysis dataset/Twitter Sentiments.csv 2021-11-06 11:57:06      3103165
extraction...
Terminé!


In [2]:
#load librairies

import numpy as np
import pandas as pd

from keras.layers import Dense, Embedding, LSTM
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences


from sklearn.model_selection import train_test_split
import re

In [3]:
#load data
df = pd.read_csv("Sentiment analysis dataset/Twitter Sentiments.csv")

In [4]:
print(df.columns)
print(df)

Index(['id', 'label', 'tweet'], dtype='object')
          id  label                                              tweet
0          1      0   @user when a father is dysfunctional and is s...
1          2      0  @user @user thanks for #lyft credit i can't us...
2          3      0                                bihday your majesty
3          4      0  #model   i love u take with u all the time in ...
4          5      0             factsguide: society now    #motivation
...      ...    ...                                                ...
31957  31958      0  ate @user isz that youuu?ðððððð...
31958  31959      0    to see nina turner on the airwaves trying to...
31959  31960      0  listening to sad songs on a monday morning otw...
31960  31961      1  @user #sikh #temple vandalised in in #calgary,...
31961  31962      0                   thank you @user for you follow  

[31962 rows x 3 columns]


In [5]:
#remove id column
new_df = df[["label", "tweet"]]
new_df

,label,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation
...,...,...
31957,0,ate @user isz that youuu?ðððððð...
31958,0,to see nina turner on the airwaves trying to...
31959,0,listening to sad songs on a monday morning otw...
31960,1,"@user #sikh #temple vandalised in in #calgary,..."


In [6]:
#preprocessing the tweets
#new_df["tweet"] = new_df["tweet"].str.lower()

#function = lambda x:re.sub('[^\w\s]','', x)
#new_df["tweet"] = new_df["tweet"].apply(function)

import gensim

new_df = df["tweet"].apply(gensim.utils.simple_preprocess)

print(new_df)

0        [user, when, father, is, dysfunctional, and, i...
1        [user, user, thanks, for, lyft, credit, can, u...
2                                  [bihday, your, majesty]
3        [model, love, take, with, all, the, time, in, ...
4                   [factsguide, society, now, motivation]
                               ...                        
31957                        [ate, user, isz, that, youuu]
31958    [to, see, nina, turner, on, the, airwaves, try...
31959    [listening, to, sad, songs, on, monday, mornin...
31960    [user, sikh, temple, vandalised, in, in, calga...
31961                 [thank, you, user, for, you, follow]
Name: tweet, Length: 31962, dtype: object


In [15]:
#initialize gensim model
modelE = gensim.models.Word2Vec(window=10,vector_size=300, min_count=1,workers=2)
print(modelE)

Word2Vec<vocab=0, vector_size=300, alpha=0.025>


In [16]:
modelE.build_vocab(new_df, progress_per=1)

In [31]:
modelE.train(new_df, total_examples=modelE.corpus_count, epochs=modelE.epochs)


(1607805, 1945025)

In [18]:
modelE.wv.most_similar("father")

[('fathers', 0.992978572845459),
 ('dad', 0.9894667863845825),
 ('fatherã', 0.9780062437057495),
 ('fathersday', 0.9708476066589355),
 ('day', 0.95847487449646),
 ('flag', 0.952197253704071),
 ('daddy', 0.9472179412841797),
 ('dads', 0.9470453858375549),
 ('heavenly', 0.9465005397796631),
 ('flagday', 0.9446587562561035)]

In [19]:
modelE.wv.vectors

array([[-1.7948069e-01,  7.2529835e-01,  1.5113737e-01, ...,
        -6.4854968e-01,  6.6579807e-01, -1.3169190e-01],
       [-1.2363248e+00,  7.7377778e-01,  7.4720308e-02, ...,
        -6.2490100e-01,  6.3512278e-01, -2.4088103e-01],
       [-8.1534433e-01,  6.5829390e-01,  1.4353622e-01, ...,
        -2.7132896e-01,  1.7079358e-01, -7.4983847e-01],
       ...,
       [-2.3325291e-03,  7.0681050e-03, -4.2100553e-04, ...,
         2.0044593e-03,  3.3791428e-03, -2.1422238e-03],
       [-6.0133054e-04,  6.0479343e-03,  2.2447591e-03, ...,
         8.2520151e-04,  3.6520901e-04,  1.6686483e-03],
       [ 1.2726227e-03,  1.0396730e-02,  1.8602500e-04, ...,
        -7.9024034e-03,  8.6711720e-03,  5.1149973e-03]], dtype=float32)

In [24]:
from tensorflow import keras

#LSTM model
embed_dim = 300
lstm_out = 196

model = keras.Sequential()
model.add(Embedding(1500, embed_dim, input_length = 31))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(loss = "binary_crossentropy", optimizer='adam', metrics = ['accuracy'])
model.summary()
          

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 31, 300)           450000    
                                                                 
 lstm_5 (LSTM)               (None, 196)               389648    
                                                                 
 dense_5 (Dense)             (None, 1)                 197       
                                                                 
Total params: 839,845
Trainable params: 839,845
Non-trainable params: 0
_________________________________________________________________


In [26]:
#encode label
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df["label"])
print("ft",y)


ft [0 0 0 ... 0 1 0]


In [28]:
print(len(modelE.wv.vectors))
print(len(y))

37890
31962


In [27]:
#divide in train-test split
x_train, x_test, y_train, y_test = train_test_split(modelE.wv.vectors, y, test_size = 0.15, random_state = 42)

y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

ValueError: ignored

In [ ]:
#train the model
model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 10, batch_size = 32)

Epoch 1/10
849/849 [==============================] - 150s 174ms/step - loss: 0.1759 - accuracy: 0.0705 - val_loss: 0.1409 - val_accuracy: 0.0682
Epoch 2/10
849/849 [==============================] - 148s 174ms/step - loss: 0.1259 - accuracy: 0.0705 - val_loss: 0.1427 - val_accuracy: 0.0682
Epoch 3/10
849/849 [==============================] - 149s 176ms/step - loss: 0.1096 - accuracy: 0.0705 - val_loss: 0.1404 - val_accuracy: 0.0682
Epoch 4/10
849/849 [==============================] - 146s 173ms/step - loss: 0.0967 - accuracy: 0.0705 - val_loss: 0.1500 - val_accuracy: 0.0682
Epoch 5/10
849/849 [==============================] - 149s 176ms/step - loss: 0.0838 - accuracy: 0.0705 - val_loss: 0.1658 - val_accuracy: 0.0682
Epoch 6/10
849/849 [==============================] - 146s 172ms/step - loss: 0.0737 - accuracy: 0.0705 - val_loss: 0.1696 - val_accuracy: 0.0682
Epoch 7/10
849/849 [==============================] - 149s 175ms/step - loss: 0.0634 - accuracy: 0.0705 - val_loss: 0.1934 -

In [ ]:
model.evaluate(x_test,y_test)



150/150 [==============================] - 5s 33ms/step - loss: 0.2592 - accuracy: 0.0682


[0.25923290848731995, 0.06819603592157364]

In [ ]:
predict_x=model.predict(x_test[1:4]) 
classes_x=np.argmax(predict_x,axis=1)

text = tokenizer.sequences_to_texts(x_test[1:4])

# for i in text :
#   print(i, end='\n')

print("dataset", text)

print("Prediction: ",classes_x)

print("Actual: \n",y_test[1:4])

1/1 [==============================] - 0s 330ms/step
dataset ['in 3 days i will be meeting my on our first lifestyle vacation', 'the using this tragedy as a way of not in my name you orlando', 'if this doesnt friday']
Prediction:  [0 0 0]
Actual: 
 [[0.]
 [0.]
 [0.]]
